<a href="https://colab.research.google.com/github/gerald-liu/credit-card-default-prediction/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Card Default Prediction
Course Project of ISOM 3360, Spring 2020

Group 21

In [ ]:
# Import basic libraries
import numpy as np 
import pandas as pd
import joblib # for saving models

In [ ]:
# For Google Colab only
# from google.colab import files

In [ ]:
# Load local data
data = pd.read_csv('data/data.csv',index_col = 'ID')

In [ ]:
# Load data from GitHub
# data_url = 'https://raw.githubusercontent.com/gerald-liu/credit-card-default-prediction/master/data/data.csv?token=AIUUIXPDCVIHEXYNIDVP23K6UEHRA'
# data = pd.read_csv(data_url, index_col = 'ID')


## Variables
There are 25 variables:

- ID: ID of each client
- LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit)
- SEX: Gender (1=male, 2=female)
- EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
- MARRIAGE: Marital status (1=married, 2=single, 3=others)
- AGE: Age in years
- PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)
- PAY_2: Repayment status in August, 2005 (scale same as above)
- PAY_3: Repayment status in July, 2005 (scale same as above)
- PAY_4: Repayment status in June, 2005 (scale same as above)
- PAY_5: Repayment status in May, 2005 (scale same as above)
- PAY_6: Repayment status in April, 2005 (scale same as above)
- BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
- BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
- BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
- BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
- BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
- BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
- PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
- PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
- PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
- PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
- PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
- PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
- default.payment.next.month: Default payment (1=yes, 0=no)


In [ ]:
data.info()

## Data Cleaning
- No missing values, but there are a few anomalous things
- Variable name:
    1. PAY_0 is very confusing, should be changed to PAY_1 in consistensy with other variable names
    2. Change target variable name from 'default.pay.next.month' to 'def_pay'
- Understand categorical and numerical data respectively


In [ ]:
# Change variable name
data = data.rename(columns={'default.payment.next.month': 'def_pay', 'PAY_0': 'PAY_1'})

In [ ]:
# To have a general idea of the default probability
data['def_pay'].value_counts()[1] / data.shape[0]

Conclusion: The dataset has a mild degree of imbalance, which is not considered as a significant problem.

In [ ]:
# Categorical variables
data['SEX'].value_counts()

In [ ]:
data['EDUCATION'].value_counts()

EDUCATION has category 5 and 6 which are 'unknown', and label 0 which is undocumented. They can all be considered as missing values.

In [ ]:
# Missing values are filled with random value according to probability of occurrence
mask_edu_good=data['EDUCATION'].isin(range(1,5))
data['EDUCATION'].mask(~mask_edu_good,data['EDUCATION'][mask_edu_good].sample(n=(~mask_edu_good).sum(),replace=True).tolist(),inplace=True)
data['EDUCATION'].value_counts()

In [ ]:
data['MARRIAGE'].value_counts()

MARRIAGE has a label 0 that is undocumented. It can be considered as missing value.

In [ ]:
mask_marriage_good=data['MARRIAGE'].isin(range(1,4))
data['MARRIAGE'].mask(~mask_marriage_good,data['MARRIAGE'][mask_marriage_good].sample(n=(~mask_marriage_good).sum(),replace=True).tolist(),inplace=True)
data['MARRIAGE'].value_counts()

One might wonder what these labels might mean something.

"Other" in education can be education lower than the high school level.

"Other" in marriage could be, for example, "divorced". 


In [ ]:
# Create lists for relevant numerical features
repayments = ['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
bill_amounts = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
payments = ['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

In [ ]:
# final check
data.info()

In [ ]:
# export to csv
data.to_csv('data/data_clean.csv')

## Data visualization

In [ ]:
data = pd.read_csv('data/data_clean.csv',index_col = 'ID')

In [ ]:
# import libraries for data visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Numerical features

#### Age

In [ ]:
data['AGE'].describe()

In [ ]:
# boxplot for age
data.boxplot(column='AGE')

Conclusion: Outliers will not be removed because they are meaningful for this feature.

In [ ]:
# boxplot for age regarding default
data.boxplot(column='AGE',by="def_pay",figsize=(5,5))

Conclusion: Age distribution is different for default clients and non-default clients, which implies that age is a significant feature.

#### Balance limit

In [ ]:
data['LIMIT_BAL'].describe()

In [ ]:
# boxplot for balance limit
data.boxplot(column='LIMIT_BAL')

Conclusion: There are a few outliers and we will apply normalization in certain models.

In [ ]:
# boxplot for balance limit regarding default
data.boxplot(column='LIMIT_BAL',by="def_pay",figsize=(5,5))

Conclusion: Balance limit distribution is different for default clients and non-default clients, whicih implies that balance limit is a significent feature.

In [ ]:
# histogram for balance limit
data.hist(column='LIMIT_BAL', figsize=(6, 3))

Conclusion: We will apply normalization in certain models.

#### Payment delay

In [ ]:
data[repayments].describe()

In [ ]:
# boxplot for payment delay
data.boxplot(column=repayments)

Conclusion: Outliers will not be removed because they are meaningful for this feature.

In [ ]:
# histogram for payment delay
data.hist(column= repayments, figsize=(12, 8))

Conclusion: Normalization is not needed because the values are clean and standard.

#### Bill amounts

In [ ]:
data[bill_amounts].describe()

In [ ]:
# boxplot for bill amounts
data.boxplot(column=bill_amounts)

Conclusion: There are a few outliers and we will apply normalization in certain models.

In [ ]:
# histogram for bill amounts
data.hist(column= bill_amounts, figsize=(12, 8))

Conclusion: We will apply normalization in certain models.

#### Preivous payments

In [ ]:
data[payments].describe()

In [ ]:
# boxplot for previous payments
data.boxplot(column=payments)

Conclusion: There are a few outliers and we will apply normalization in certain models.

In [ ]:
# histogram for preious payments
data.hist(column= payments, figsize=(12, 8))

Conclusion: We will apply normalization in certain models.

### Categorial features

In [ ]:
# age_bins = pd.cut(data['AGE'], 10, retbins=True)
# age_bins = np.floor(age_bins).astype(int)
# df_male = data[data['SEX'] == 1]
# df_female = data[data['SEX'] == 2]

# prob_male = df_male.groupby(pd.cut(df_male['AGE'], bins=age_bins))['def_pay'].mean()
# prob_female = df_female.groupby(pd.cut(df_female['AGE'], bins=age_bins))['def_pay'].mean()

# plt.figure(figsize=(18, 6))
# plt.title('Default rate for different age groups')
# plt.bar(range(0, 30, 3), prob_male, width=1, label='Male')
# plt.bar(range(1, 31, 3), prob_female, width=1, label='Female')
# plt.xticks(range(0, 30, 3), prob_male.index)
# plt.legend(loc='upper left')
# plt.show()

In [ ]:
# Sex
# histogram for sex
g1 = sns.FacetGrid(data, col='SEX', row='def_pay', margin_titles=True)
g1.map(plt.hist,'AGE',color='blue')

Conclusion: Sex is a significant feature, and we can infer that male are more likely to default than female.

In [ ]:
# Education
# histogram for education
g2 = sns.FacetGrid(data,row='def_pay', margin_titles=True)
g2.map(plt.hist,'EDUCATION',color='blue')

Conclusion: Education is a significant feature, and we can infer that people with higher education level are less likely to default.

In [ ]:
# MARRIAGE
# histogram for marriage
g3 = sns.FacetGrid(data,row='def_pay', margin_titles=True)
g3.map(plt.hist,'MARRIAGE',color='blue')

Conclusion: Marriage is a significant feature, and we can infer that married people are more likely to default than single people.

## Model Building

### Decision tree

In [ ]:
# import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

#### With feature engineering

In [ ]:
data = pd.read_csv('data/data_clean.csv',index_col = 'ID')

In [ ]:
# binning of age
group_age=['YOUNG','MIDDLE','OLD']
data['AGE-BINNED']=pd.cut(data['AGE'],3,labels=group_age)
data['AGE-BINNED'].value_counts().plot(kind='bar')

In [ ]:
# label encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['AGE-BINNED'] = le.fit_transform(data['AGE-BINNED'])

We will use min-max scaling for numerical features. As there are meaningful negative values, log scaling should not be used. The distribution is not normal, so z-score should not be used. Feature clipping will clip a few values which we believe are in fact normal points. Thus, we choose min-max scaling.

In [ ]:
# normalization of certain numerical features
numerical_features_featureengineering = ['LIMIT_BAL','BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6',
                                         'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(data[numerical_features_featureengineering])
data[numerical_features_featureengineering] = scaler.transform(data[numerical_features_featureengineering])

In [ ]:
# (log scaling)

# numerical_features_featureengineering = ['LIMIT_BAL','BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6',
#                                          'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']
# data[numerical_features_featureengineering]=data[numerical_features_featureengineering].transform(lambda x:np.log(x+1))

# from sklearn.preprocessing import FunctionTransformer
# transformer = FunctionTransformer(np.log1p, validate=True)
# data[numerical_features_featureengineering]=transformer.transform(data[numerical_features_featureengineering])

In [ ]:
# data.isnull().any()

In [ ]:
data.to_csv('data/data_decisiontree.csv')

In [ ]:
data = pd.read_csv('data/data_decisiontree.csv',index_col = 'ID')

In [ ]:
features=['LIMIT_BAL','SEX','EDUCATION','MARRIAGE','AGE-BINNED'] + repayments + bill_amounts + payments
target=['def_pay']

In [ ]:
X1=data[features]
y1=data[target]

In [ ]:
DTM1_0 = DecisionTreeClassifier()
DTM1_0.fit(X1,y1)

In [ ]:
DTM1_0.get_depth()

In [ ]:
DTM1_0.get_n_leaves()

In [ ]:
# import cross validation
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
score_cv_dt = cross_val_score(DTM1_0, X1, y1, cv=10)

In [ ]:
score_cv_dt.mean()

In [ ]:
# define the numbers to try out for depth and max number of leaf nodes 
depths = list(range(5, 45, 5))
num_leafs = list(range(500, 5000, 500))

In [ ]:
param_grid = [{'max_depth':depths,
              'max_leaf_nodes':num_leafs}]

In [ ]:
# define the model using GridSearchCV
DTM1 = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid, cv=10)

In [ ]:
DTM1.fit(X1,y1)

In [ ]:
# find the best parameters
DTM1.best_params_

In [ ]:
DTM1.best_score_

In [ ]:
DTM1_best = DTM1.best_estimator_

In [ ]:
joblib.dump(DTM1_best, 'models/DTM1_best.joblib')

In [ ]:
# For Google Colab only
# files.download('models/DTM1_best.joblib') 

In [ ]:
# For Google Colab only
# DTM1_file = files.upload()

In [ ]:
DTM1_best = joblib.load('models/DTM1_best.joblib')

#### Without feature engineering

In [ ]:
data = pd.read_csv('data/data_clean.csv',index_col = 'ID')

In [ ]:
# define independent variables / attirbutes / features
features = ['LIMIT_BAL','SEX','EDUCATION','MARRIAGE','AGE'] + repayments + bill_amounts + payments
# define one single target variable / label
target = ['def_pay']

In [ ]:
# get defined training dataset
X2 = data[features]
y2 = data[target]

In [ ]:
# build a decision tree model with the defalut hyperparameter settings
DTM2_0 = DecisionTreeClassifier()
DTM2_0.fit(X2,y2)

In [ ]:
DTM2_0.get_depth()

In [ ]:
DTM2_0.get_n_leaves()

In [ ]:
# import cross validation
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
# CV
score_cv_dt = cross_val_score(DTM2_0, X2, y2, cv=10)

In [ ]:
score_cv_dt.mean()

In [ ]:
# define the numbers to try out for depth and max number of leaf nodes 
depths = list(range(5, 45, 5))
num_leafs = list(range(500, 5000, 500))

In [ ]:
param_grid = [{'max_depth':depths,
              'max_leaf_nodes':num_leafs}]

In [ ]:
# define the model using GridSearchCV
DTM2 = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid, cv=10)

In [ ]:
DTM2.fit(X2,y2)

In [ ]:
# find the best parameters
DTM2.best_params_

In [ ]:
DTM2.best_score_

In [ ]:
DTM2_best = DTM2.best_estimator_

In [ ]:
joblib.dump(DTM2_best, 'models/DTM2_best.joblib')

In [ ]:
# For Google Colab only
# files.download('models/DTM2_best.joblib') 

In [ ]:
# For Google Colab only
# DTM2_file = files.upload()

In [ ]:
DTM2_best = joblib.load('models/DTM2_best.joblib')

### Logistic regression

#### If we do not conduct scaling on numerical features, logistic regression will give a convergence warning. So we choose to do mix-max scaling for numerical features and need to do one-hot encoding for categorical features.

In [ ]:
data = pd.read_csv('data/data_clean.csv',index_col = 'ID')

In [ ]:
# the categorical features to carry out feature engineering
categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']
# one-hot encoding
data = pd.get_dummies(data, columns=categorical_features,drop_first = True)

In [ ]:
# the numerical features to carry out feature engineering
numerical_features_featureengineering = ['AGE','LIMIT_BAL','PAY_1','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6',
                                         'BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6',
                                         'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(data[numerical_features_featureengineering])
data[numerical_features_featureengineering] = scaler.transform(data[numerical_features_featureengineering])

In [ ]:
# Export preprocessed data
data.to_csv('data/data_logistic.csv')

In [ ]:
# For Google Colab only
# files.download('data/data_logistic.csv') 

In [ ]:
# Read preprocessed data from local drive
data = pd.read_csv('data/data_logistic.csv',index_col = 'ID')

In [ ]:
# Read preprocessed data from GitHub
# data_logistc_url = 'https://raw.githubusercontent.com/gerald-liu/credit-card-default-prediction/master/data/data_clean.csv?token=AIUUIXPDCVIHEXYNIDVP23K6UEHRA'
# data = pd.read_csv(data_logistic_url, index_col = 'ID')

In [ ]:
X = data.drop(columns='def_pay')

In [ ]:
# import Logistic Regression from sklearn
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(penalty='l1', solver='saga',max_iter=1000)

In [ ]:
lr

In [ ]:
# change target column to array
y_act = y.values.ravel()

In [ ]:
score_cv_lr = cross_val_score(lr, X, y_act, cv=10)

In [ ]:
score_cv_lr.mean()

In [ ]:
joblib.dump(lr, 'models/Logistic.joblib')

In [ ]:
# For Google Colab only
# files.download('models/Logistic.joblib') 

In [ ]:
# For Google Colab only
# lr_file = files.upload()

In [ ]:
lr = joblib.load('models/Logistic.joblib')

### K-means clustering

In [ ]:
data = pd.read_csv('data/data_clean.csv',index_col = 'ID')

In [ ]:
# the categorical features to carry out feature engineering
categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']

# the numerical features to carry out feature engineering
numerical_features_featureengineering = ['AGE','LIMIT_BAL','PAY_1','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']

In [ ]:
# one-hot encoding
data = pd.get_dummies(data, columns=categorical_features,drop_first = True)

In [ ]:
# Normalize attributes
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler().fit(data[numerical_features_featureengineering])
data[numerical_features_featureengineering] = scaler.transform(data[numerical_features_featureengineering])

In [ ]:
# Export preprocessed data
data.to_csv('data/data_kmeans.csv')

In [ ]:
# Read preprocessed data from local drive
data = pd.read_csv('data/data_kmeans.csv',index_col = 'ID')

In [ ]:
X = data.drop(columns='def_pay')
y = data['def_pay']

In [ ]:
# import KMeans from sklearn
from sklearn.cluster import KMeans

In [ ]:
# set number of clusters
kmeansmodel = KMeans(n_clusters=2)

In [ ]:
# fit data
kmeansmodel.fit(X)

In [ ]:
# centroids
centroids = kmeansmodel.cluster_centers_
centroids

In [ ]:
# centroids
interation = kmeansmodel.n_iter_
interation

In [ ]:
# SSD
SSD = kmeansmodel.inertia_
SSD

In [ ]:
# centroids
distance = kmeansmodel.transform(X)
distance

In [ ]:
# Get Labels of each point 
data_label = kmeansmodel.labels_

In [ ]:
# Create comparison table between actual def_pay and predict cluster
data_comparison=pd.DataFrame(data_label,y,columns=['cluster'])
data_comparison

In [ ]:
# Find corresponding instances that belong to each def_pay type
data_nondefault = data_comparison.loc[0]
data_default = data_comparison.loc[1]

In [ ]:
# Count value for different cluster regarding one type of def_pay
data_nondefault[data_nondefault.columns[0]].value_counts()

In [ ]:
data_default[data_default.columns[0]].value_counts()

Based on majority rule, cluster 0 is default, cluster 1 is nondefault.

## Model Evaluation

In [ ]:
# import evaluation tools
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

### Decision tree

#### With feature engineering

In [ ]:
DTM1_pred = DTM1_best.predict(X1)

In [ ]:
# Confusion matrix
print("Confusion Matrix:", '\n',confusion_matrix(y1, DTM1_pred))
print("Accuracy:",accuracy_score(y1, DTM1_pred, normalize=True, sample_weight=None))

In [ ]:
print(classification_report(y1, DTM1_pred))

In [ ]:
# draw roc

# scores=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
# fpr, tpr, thresholds = roc_curve(y1, scores, pos_label=1)

#### Without feature engineering

In [ ]:
DTM2_pred = DTM2_best.predict(X2)

In [ ]:
# Confusion matrix
print("Confusion Matrix:", '\n',confusion_matrix(y2, DTM2_pred))
print("Accuracy:",accuracy_score(y2, DTM2_pred, normalize=True, sample_weight=None))

In [ ]:
print(classification_report(y2,DTM2_pred))

In [ ]:
# draw roc


### Logistic regression

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
# predict value of target based on cross validation
lr_pred = cross_val_predict(lr, X, y_act, cv=10)

In [ ]:
# Confusion matrix
print("Confusion Matrix:", '\n',confusion_matrix(y, lr_pred))
print("Accuracy:",accuracy_score(y, lr_pred, normalize=True, sample_weight=None))

In [ ]:
print(classification_report(y, lr_pred))

In [ ]:
# draw roc


### K-means clustering

In this case, the cardinality is fixed due to that the number of clusters is fixed, so the way of checking if cardinality correlates with magnitude cannot be used. Thus, the only way is to interpret the above results, which kind of serve as a "confusion matrix".

For non default clients, 12509 are clustered correctly and 10855 wrongly. For default clients, 3346 are clustered correctly and 3290 wrongly. "Accuracy score" is around 0.5285.